In [11]:
import pandas as pd
import ipaddress
from intervaltree import Interval, IntervalTree
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\HP\10 Acadamy PRojects\New folder (8 & 9)\Ehanced_Fraud_Detection_Ecommerce_Banking\notebooks


In [4]:

# Function to convert IP address to integer
def ip_to_int(ip):
    if pd.isna(ip):
        return None
    try:
        return int(ip)
    except ValueError:
        return int(ipaddress.ip_address(ip))


In [14]:
def main():
    # Load the datasets
    fraud_data = pd.read_csv('../data/raw/Fraud_Data.csv')
    ip_address_data = pd.read_csv('../data/raw/ipAddress_to_Country.csv')

    # Print columns of ip_address_data to inspect
    print(ip_address_data.columns)

    # Handle NaN values in IP address columns and convert IP address ranges to integers
    ip_address_data.dropna(subset=['lower_bound_ip_address', 'upper_bound_ip_address'], inplace=True)
    ip_address_data['lower_bound_ip_address_int'] = ip_address_data['lower_bound_ip_address'].apply(ip_to_int)
    ip_address_data['upper_bound_ip_address_int'] = ip_address_data['upper_bound_ip_address'].apply(ip_to_int)

    # Create an interval tree for IP ranges
    ip_tree = IntervalTree()
    for _, row in ip_address_data.iterrows():
        ip_tree[row['lower_bound_ip_address_int']:row['upper_bound_ip_address_int'] + 1] = row['country']

    # Function to map IP address to country using the interval tree
    def map_ip_to_country(ip_int):
        if ip_int is None:
            return 'Unknown'
        interval = ip_tree[ip_int]
        if interval:
            return interval.pop().data
        else:
            return 'Unknown'

    # Handle NaN values in fraud_data IP addresses and apply function to get country for each IP address
    fraud_data.dropna(subset=['ip_address'], inplace=True)
    fraud_data['ip_address_int'] = fraud_data['ip_address'].apply(ip_to_int)
    fraud_data['country'] = fraud_data['ip_address_int'].apply(map_ip_to_country)

    # Save the processed fraud data with country information
    fraud_data.to_csv('../data/raw/merged_fraud_data_with_country.csv', index=False)

    # Display the first few rows of the updated fraud_data
    print(fraud_data.head())

if __name__ == "__main__":
    main()
    

Index(['lower_bound_ip_address', 'upper_bound_ip_address', 'country'], dtype='object')
   user_id          signup_time        purchase_time  purchase_value  \
0    22058  2015-02-24 22:55:49  2015-04-18 02:47:11              34   
1   333320  2015-06-07 20:39:50  2015-06-08 01:38:54              16   
2     1359  2015-01-01 18:52:44  2015-01-01 18:52:45              15   
3   150084  2015-04-28 21:13:25  2015-05-04 13:54:50              44   
4   221365  2015-07-21 07:09:52  2015-09-09 18:40:53              39   

       device_id source browser sex  age    ip_address  class  ip_address_int  \
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0       732758368   
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0       350311387   
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2.621474e+09      1      2621473820   
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0      3840542443   
4  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0       4155831